In [1]:
from pathlib import Path

SURVEY_DIR = Path("/media/ko/MAUI63/2025-08-maui-winter-survey")
OUTPUT_DIR = Path("/workspaces/cv/survey-review-app/data")

In [6]:
# Get all the frames:
from collections import namedtuple
from datetime import datetime, timedelta
import pytz

GpsTriggerEvent = namedtuple("GpsTriggerEvent", ["time", "lat", "lon", "alt", "ignore", "line_idx"])

# Parse the GPS data:
gps_points = []
for fpath in sorted(SURVEY_DIR.rglob("*.CAM")):
    # Parse the start time from the filename:
    # Read the files:
    with open(fpath, "r") as f:
        for line_idx, line in enumerate(f):
            parts = line.split(",")
            week = int(parts[0])
            start_time = datetime(1980, 1, 6, tzinfo=pytz.utc) + timedelta(weeks=week)
            milliseconds_since_start_of_week = float(parts[1])
            num_satellites = int(parts[7])
            ignore = False
            if num_satellites < 12:
                print(f"Ignoring GPS point with {num_satellites} satellites")
                ignore = True
                continue
            lat = float(parts[8])
            lon = float(parts[9])
            alt = float(parts[10])
            time = start_time + timedelta(milliseconds=milliseconds_since_start_of_week)
            gps_points.append(GpsTriggerEvent(time.isoformat(), lat, lon, alt, ignore, line_idx))


Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
I

In [ ]:
import json

with open("./data/frames.json") as f:
    frames = json.load(f)
frames[0]

{'lat': -37.807251549,
 'lon': 174.837439757,
 'alt': 752.6893,
 'gps_time': '2025-08-11T22:06:13.129000+00:00',
 'images': {'l09': {'path': '2025-08-12/cameras/l09/DCIM/10050812/_09L4188.JPG',
   't': '2025-08-12T10:06:12.726765+12:00',
   'bounds_lbrt': [[174.83460080877316, -37.80815247380174],
    [174.8346008777554, -37.80635055564688],
    [174.837439757, -37.80635058985239],
    [174.837439757, -37.80815250800946]]},
  'l28': {'path': '2025-08-12/cameras/l28/DCIM/10050812/_28L4173.JPG',
   't': '2025-08-12T10:06:12.709647+12:00',
   'bounds_lbrt': [[174.83119407090834, -37.80815234244409],
    [174.83119422266924, -37.806350424297705],
    [174.8346008777554, -37.80635055564688],
    [174.83460080877316, -37.80815247380174]]},
  'r09': {'path': '2025-08-12/cameras/r09/DCIM/10050812/_09R4136.JPG',
   't': '2025-08-12T10:06:12.889059+12:00',
   'bounds_lbrt': [[174.837439757, -37.80815250800946],
    [174.837439757, -37.80635058985239],
    [174.8402786362446, -37.80635055564688],

In [7]:
gps_points[0]

GpsTriggerEvent(time='2025-08-11T21:22:29.716000+00:00', lat=-37.864363521, lon=175.336249145, alt=51.4326, ignore=False, line_idx=0)

In [ ]:
from tqdm import tqdm


unmatched = []
assert len(set(f["gps_time"] for f in frames)) == len(frames), "gps_time should be unique in frames"
lookup = {f["gps_time"]: f for f in frames}
for g in tqdm(gps_points):
    matched = lookup.get(g.time)
    if not matched:
        unmatched.append(g)
print(len(unmatched), "unmatched GPS points out of", len(gps_points))


100%|██████████| 39485/39485 [00:00<00:00, 2639068.32it/s]

99 unmatched GPS points out of 39485


In [ ]:
# Add them:
for g in unmatched:
    frames.append({"lat": g.lat, "lon": g.lon, "alt": g.alt, "gps_time": g.time, "images": {}, "line_idx": None})
frames = sorted(frames, key=lambda f: f["gps_time"])
frames[0]

{'lat': -37.864363521,
 'lon': 175.336249145,
 'alt': 51.4326,
 'gps_time': '2025-08-11T21:22:29.716000+00:00',
 'images': {},
 'line_idx': None}

In [12]:
with open(Path("./data/frames.json"), "w") as f:
    json.dump(frames, f)

In [8]:
# Get all the photos:
import json


camera_photos = {}
processed_frames = []
possible_detections = []
for daydir in sorted(SURVEY_DIR.iterdir()):
    if not daydir.is_dir():
        continue

    # Load GPS matched stuff:
    print("Reading ", daydir / "processed_frames.json")
    with open(daydir / "processed_frames.json") as f:
        frames = json.load(f)["frames"]
    # Add full path
    for f in frames:
        for i in f["images"].values():
            i["path"] = f"{daydir.name}/{i['path']}"
    processed_frames.extend(frames)

    # Load possible detections
    for camera_dir in (daydir / "inferences").iterdir():
        for reviewed_img in (camera_dir / "reviewed").rglob("*.jpg"):
            relpath = reviewed_img.relative_to(camera_dir / "reviewed")
            inferences_path = camera_dir / "inferences" / relpath.with_suffix(".json")
            with open(inferences_path) as f:
                detections = json.load(f)
            assert len(detections) > 0
            img_path = daydir / "cameras" / camera_dir.name / "DCIM" / relpath.with_suffix(".JPG")
            assert img_path.exists(), f"Image path {img_path} does not exist"
            possible_detections.append(
                {
                    "camera": camera_dir.name,
                    "img_path": img_path,
                    "detections": detections,
                }
            )

    # Load frames
    cameras_dir = daydir / "cameras"
    for camera_dir in sorted(cameras_dir.iterdir()):
        camera_name = camera_dir.name
        print(f"Getting photos from {camera_dir}")
        for img_path in camera_dir.rglob("*.JPG"):
            if camera_name not in camera_photos:
                camera_photos[camera_name] = []
            camera_photos[camera_name].append(img_path)

Reading  /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-12/processed_frames.json
Getting photos from /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-12/cameras/l09
Getting photos from /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-12/cameras/l28
Getting photos from /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-12/cameras/r09
Getting photos from /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-12/cameras/r28
Reading  /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-13/processed_frames.json
Getting photos from /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-13/cameras/l09
Getting photos from /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-13/cameras/l28
Getting photos from /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-13/cameras/r09
Getting photos from /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-13/cameras/r28
Reading  /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-14/processed_frames.json
Getting photos from /media/ko/MAUI63/2025-08-m

In [9]:
print(f"Loaded {len(processed_frames)} gps points")
print(f"Loaded {sum(len(v) for v in camera_photos.values())} photos")
print(f"Loaded {len(possible_detections)} possible detections")

Loaded 39386 gps points
Loaded 160829 photos
Loaded 33 possible detections


In [10]:
# OK, for each gps point, get the bounding box of each image so we can show it on a map
import pyproj

geodesic = pyproj.Geod(ellps="WGS84")

# TODO: calculate offsets from altitude
offsets = {
    "l28": (-550, -250),
    "l09": (-250, 0),
    "r09": (0, 250),
    "r28": (250, 550),
}
image_height = 200
sorted_frames = sorted(processed_frames, key=lambda x: x["gps_time"])
for idx in range(len(sorted_frames)):
    p1 = sorted_frames[idx]
    bearing = 0
    if idx > 0:
        p0 = sorted_frames[idx - 1]
        bearing, _, _ = geodesic.inv(p0["lon"], p0["lat"], p1["lon"], p1["lat"])

    for cam, d in p1["images"].items():
        x0, x1 = offsets[cam]
        forward_lon, forward_lat, _ = geodesic.fwd(p1["lon"], p1["lat"], bearing, image_height / 2)
        rear_lon, rear_lat, _ = geodesic.fwd(p1["lon"], p1["lat"], bearing + 180, image_height / 2)
        angle = bearing - 90 if x0 < 0 else bearing + 90
        top_left_lon, top_left_lat, _ = geodesic.fwd(forward_lon, forward_lat, angle, abs(x0))
        top_right_lon, top_right_lat, _ = geodesic.fwd(forward_lon, forward_lat, angle, abs(x1))
        bottom_left_lon, bottom_left_lat, _ = geodesic.fwd(rear_lon, rear_lat, angle, abs(x0))
        bottom_right_lon, bottom_right_lat, _ = geodesic.fwd(rear_lon, rear_lat, angle, abs(x1))

        d["bounds_lbrt"] = [
            [bottom_left_lon, bottom_left_lat],
            [top_left_lon, top_left_lat],
            [top_right_lon, top_right_lat],
            [bottom_right_lon, bottom_right_lat],
        ]

processed_frames[0]

{'lat': -37.807251549,
 'lon': 174.837439757,
 'alt': 752.6893,
 'gps_time': '2025-08-11T22:06:13.129000+00:00',
 'images': {'l09': {'path': '2025-08-12/cameras/l09/DCIM/10050812/_09L4188.JPG',
   't': '2025-08-12T10:06:12.726765+12:00',
   'bounds_lbrt': [[174.83460080877316, -37.80815247380174],
    [174.8346008777554, -37.80635055564688],
    [174.837439757, -37.80635058985239],
    [174.837439757, -37.80815250800946]]},
  'l28': {'path': '2025-08-12/cameras/l28/DCIM/10050812/_28L4173.JPG',
   't': '2025-08-12T10:06:12.709647+12:00',
   'bounds_lbrt': [[174.83119407090834, -37.80815234244409],
    [174.83119422266924, -37.806350424297705],
    [174.8346008777554, -37.80635055564688],
    [174.83460080877316, -37.80815247380174]]},
  'r09': {'path': '2025-08-12/cameras/r09/DCIM/10050812/_09R4136.JPG',
   't': '2025-08-12T10:06:12.889059+12:00',
   'bounds_lbrt': [[174.837439757, -37.80815250800946],
    [174.837439757, -37.80635058985239],
    [174.8402786362446, -37.80635055564688],

In [11]:
from PIL import Image

# Convert detection sizes to x,y,w,h relative:
for det in possible_detections:
    path = det["img_path"]
    with Image.open(path) as img:
        iw, ih = img.size
    for d in det["detections"]:
        x0, y0, x1, y1 = d["x0"], d["y0"], d["x1"], d["y1"]
        left = x0 / iw
        top = y0 / ih
        width = (x1 - x0) / iw
        height = (y1 - y0) / ih
        d["ltwh"] = [left, top, width, height]
possible_detections[0]

{'camera': 'l09',
 'img_path': PosixPath('/media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-13/cameras/l09/DCIM/10150813/_09L0183.JPG'),
 'detections': [{'x0': 120.07931518554688,
   'y0': 2048.131591796875,
   'x1': 165.4490966796875,
   'y1': 2109.912353515625,
   'label': 'maui-or-hectors',
   'score': 0.5148220062255859,
   'ltwh': [0.01263460807928734,
    0.3232530921396583,
    0.004773756470343079,
    0.009750751533893624]},
  {'x0': 5120,
   'y0': 1624.9957122802734,
   'x1': 5131.011950492859,
   'y1': 1650.07958984375,
   'label': 'maui-or-hectors',
   'score': 0.023116517812013626,
   'ltwh': [0.5387205387205387,
    0.2564702828725179,
    0.0011586648245853206,
    0.003958945322518397]}]}

In [12]:
import json

# Save it out:
with open(OUTPUT_DIR / "frames.json", "w") as f:
    json.dump(processed_frames, f, indent=2)
with open(OUTPUT_DIR / "camera_photos.json", "w") as f:
    json.dump({k: [str(p.relative_to(SURVEY_DIR)) for p in v] for k, v in camera_photos.items()}, f, indent=2)
with open(OUTPUT_DIR / "possible_detections.json", "w") as f:
    json.dump(
        [
            {
                "camera": d["camera"],
                "img_path": str(d["img_path"].relative_to(SURVEY_DIR)),
                "detections": d["detections"],
            }
            for d in possible_detections
        ],
        f,
        indent=2,
    )
# Get the first image from camera l28
